# 1. Подготовка данных

In [1]:
# Источник и описание полей датасета: http://archive.ics.uci.edu/ml/datasets/online+retail#

In [2]:
# Импортируем нужные библиотеки
import pandas as pd
import numpy as np

In [3]:
# Загружаем и читаем датасет
# df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
# Смотрим размеры датасета
df.shape

(541909, 8)

In [5]:
# Чистим датасет: удаляем записи с нулевыми покупками
df = df.loc[df['Quantity'] > 0]

In [6]:
# Чистим датасет: удаляем записи с пропусками в customer_id
df = df.dropna(subset=['CustomerID'])

In [7]:
# Агрегируем данные в сводную таблицу "покупатель - товар"
customer_item_matrix = df.pivot_table(index='CustomerID', columns='StockCode', values='Quantity', aggfunc='sum')
customer_item_matrix.head()

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12347.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12348.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
12349.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [8]:
# Смотрим на размеры таблицы
customer_item_matrix.shape

(4339, 3665)

In [9]:
# Сравниваем кол-во строк с кол-вом уникальных покупателей в исходном датафрейме
df['CustomerID'].nunique()

4339

In [10]:
# Сравниваем кол-во столбцов с кол-вом уникальных товаров в исходном датафрейме
df['StockCode'].nunique()

3665

In [11]:
# Заменяем кол-во покупок на событие: 1- факт покупки, 0 - отсутствие покупки
customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0)
customer_item_matrix.head()

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12349.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12350.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


#2. Коллаборативная фильтрация (Collaborative Filtering)

In [12]:
a = np.array([1, 1, 2])

In [13]:
# Преобразуем исходную матрицу "покупатель - товар" в массив numpy
m = customer_item_matrix.values
m

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
# Импортируем библиотеку для создания разреженной матрицы
from scipy.sparse import csr_matrix

In [15]:
# Задаем функцию расчета косинуса между векторами-строками товаров
def cos_sim(array):
  csr_array = csr_matrix(array)
  numerator = np.dot(csr_array.T,csr_array).toarray()
  denomenator = np.linalg.norm(array.T, axis = 1, keepdims = True)*np.linalg.norm(array, axis = 0, keepdims = True)
  return numerator / denomenator

In [16]:

item_item_sim_matrix = cos_sim(m)
item_item_sim_matrix

array([[1.        , 0.        , 0.09486833, ..., 0.06691496, 0.        ,
        0.07821652],
       [0.        , 1.        , 0.        , ..., 0.01618174, 0.        ,
        0.        ],
       [0.09486833, 0.        , 1.        , ..., 0.07053456, 0.        ,
        0.01099299],
       ...,
       [0.06691496, 0.01618174, 0.07053456, ..., 1.        , 0.        ,
        0.0775386 ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.07821652, 0.        , 0.01099299, ..., 0.0775386 , 0.        ,
        1.        ]])

In [17]:
# Добавляем название столбцов и индексов в матрицу мер близости между векторами-строками товаров
item_item_sim_matrix = pd.DataFrame(item_item_sim_matrix)

item_item_sim_matrix.columns = customer_item_matrix.columns

item_item_sim_matrix['StockCode'] = customer_item_matrix.columns
item_item_sim_matrix = item_item_sim_matrix.set_index('StockCode')

item_item_sim_matrix.head()

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.000000,0.094868,0.090351,0.062932,0.098907,0.095346,0.047673,0.075593,0.090815,...,0.0,0.0,0.0,0.0,0.0,0.029361,0.0,0.066915,0.000000,0.078217
10080,0.000000,1.000000,0.000000,0.032774,0.045655,0.047836,0.000000,0.000000,0.082261,0.049413,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.016182,0.000000,0.000000
10120,0.094868,0.000000,1.000000,0.057143,0.059702,0.041703,0.060302,0.060302,0.095618,0.028718,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.010993
10125,0.090351,0.032774,0.057143,1.000000,0.042644,0.044682,0.043073,0.000000,0.051224,0.030770,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.070669
10133,0.062932,0.045655,0.059702,0.042644,1.000000,0.280097,0.045002,0.060003,0.071358,0.057152,...,0.0,0.0,0.0,0.0,0.0,0.036955,0.0,0.070185,0.049752,0.021877


# 3. Топ 10 похожих товаторов для товара 10002 -  	INFLATABLE POLITICAL GLOBE

In [31]:
top10 = item_item_sim_matrix.loc[10002].sort_values(ascending=False).iloc[1:11]
top10

StockCode
90103     0.223607
21826     0.197642
90101     0.182574
90059F    0.182574
90059E    0.182574
90059C    0.182574
90059B    0.182574
16010     0.182574
84535A    0.181369
21439     0.177123
Name: 10002, dtype: float64

In [ ]:
df[df[]]

In [42]:
# Выводим перечень товаров, рекомендованных для нашего эталонного покупателя
asnw = df.loc[df['StockCode'].isin(top10.index), ['StockCode', 'Description']].drop_duplicates().set_index('StockCode')
answ = asnw.join(top10, lsuffix='StockCode', rsuffix='StockCode', how = 'inner').sort_values(by = 10002, ascending = False)
answ[['Description']]

,Description
StockCode,
90103,PURPLE FRANGIPANI NECKLACE
21826,EIGHT PIECE DINOSAUR SET
90059B,DIAMANTE HAIR GRIP PACK/2 BLACK DIA
90059E,DIAMANTE HAIR GRIP PACK/2 RUBY
90059C,DIAMANTE HAIR GRIP PACK/2 MONTANA
16010,FOLDING CAMPING SCISSOR W/KNIF & S
90059F,DIAMANTE HAIR GRIP PACK/2 LT ROSE
90101,WHITE FRANGIPANI NECKLACE
84535A,ENGLISH ROSE NOTEBOOK A6 SIZE
